<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/TunningCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloads e imports

In [ ]:
# !pip install tensorflow
# !pip install keras
# !pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import sklearn
import numpy as np
import pandas as pd
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.layers import Dropout, Conv1D, MaxPooling1D, Flatten, Dense, SpatialDropout1D, BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import schedules, Adam, Adadelta, SGD, RMSprop, Adagrad, Adamax, Nadam, Ftrl # Estamos usando só o Nadam
from keras.models import Sequential
from keras.layers.embeddings import Embedding

from mlxtend.plotting import plot_confusion_matrix
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

import keras_tuner

In [ ]:
def tokenize(tweets_list):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweets_list)
    return tokenizer.texts_to_sequences(tweets_list), tokenizer

In [ ]:
def pad(text_tokenized, length=None):
    return pad_sequences(text_tokenized, maxlen=length, padding='post')

In [ ]:
def preprocess(tweets_list, max_text_length):
   
    preprocess_tweets_list, tweets_list_tokenizer = tokenize(tweets_list)

    preprocess_tweets_list = pad(preprocess_tweets_list, length=max_text_length)

    return preprocess_tweets_list, tweets_list_tokenizer

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            self.score = roc_auc_score(self.y_val, y_pred)
            self.false_positive_rate, self.true_positive_rate, _ = roc_curve(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, self.score))             

# Carrega arquivo de saída (resultados)

In [ ]:
# CONSTANTES DOS RESULTADOS

REDE_CNN = 'CNN'
REDE_LSTM = 'LSTM'
REDE_BILSTM = 'Bi-LSTM'

BASE_1 = 'BASE 1'
BASE_2 = 'BASE 2'

FUNCAO_ATV_SIGMOID = 'SIGMOID'
FUNCAO_ATV_SOFTMAX = 'SOFTMAX'
FUNCAO_ATV_RELU = 'RELU'

OTIMIZADOR_1 = '1'
OTIMIZADOR_2 = '2'
OTIMIZADOR_3 = '3'
OTIMIZADOR_4 = '4'

# Caminho arquivo de saída
PATH_ARQ_SAIDA = "/content/drive/MyDrive/TCC/resultados/resultados.csv"

resultados = pd.read_csv(PATH_ARQ_SAIDA, index_col=0)
resultados.head()

,rede,base,funcao_ativacao,otimizador,acuracia,roc_curve,train_resume,confusion_matrix
0,CNN,BASE 2,SIGMOID,1,0.919622,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 954, ""01"": 93, ""10"": 77, ""11"": 991}"
1,CNN,BASE 2,SIGMOID,1,0.925768,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 983, ""01"": 64, ""10"": 93, ""11"": 975}"
2,CNN,BASE 2,SIGMOID,1,0.928605,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 970, ""01"": 77, ""10"": 74, ""11"": 994}"
3,CNN,BASE 2,SIGMOID,1,0.921513,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 952, ""01"": 95, ""10"": 71, ""11"": 997}"
4,CNN,BASE 2,SIGMOID,1,0.927187,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 974, ""01"": 73, ""10"": 81, ""11"": 987}"


# Carrega base

In [ ]:
# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"
#MATRIZ_CBOW_300_BASE_1 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base1JuntaHateENotHateDaLeila_balanceada_CBOW300.CSV", delimiter=',')

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"
#MATRIZ_CBOW_300_BASE_2_LEILA = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_maior4_menor25_limpissima_balanceada_CBOW300.CSV", delimiter=',')

PATH_BASE_1_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1_classificada_balanceada.csv" #entrada

PATH_BASE_2_CLASSIFICADA_BALANCEADA = r"/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv" #entrada
MATRIZ_CBOW_300_BASE_2 = loadtxt("/content/drive/MyDrive/TCC/dados/word_embeddings/Matriz_Base2_classificada_balanceada_CBOW300.csv", delimiter=',') #saida

# Execução modelo

In [ ]:
tweets = pd.read_csv(PATH_BASE_2_CLASSIFICADA_BALANCEADA, index_col=0)
text_column = tweets['text']
text_column

0        retwet bahia fazer sendo governada pt sei baia...
1        k imagine atitude mental negativa f idiota fec...
2        general heleno é bolsonaro rosna late late lat...
3        alan ser cara pau achar havendo ruptura nesse ...
4        canalhascomunistas caçarão chapa presidencialn...
                               ...                        
10569               af hein amiga mandar outro número mail
10570    egoísmo é grande parte produto sociedade émile...
10571    abençoado novo linda querida obrigada carinho ...
10572                              leva gente amorzinhos d
10573                                   k k ata meia sonsa
Name: text, Length: 10574, dtype: object

In [ ]:
# Pega a média de caracteres dos tweets de toda a base
max_text_length = int(text_column.apply(lambda x: len(str(x).split(' '))).max())
max_text_length

55

In [ ]:
output_label = tweets['label']
input_data, text_tokenizer = preprocess(text_column, None)
    
text_vocab_size = len(text_tokenizer.word_index)
print("Vocabulary size:", text_vocab_size)

Vocabulary size: 19043


In [ ]:
# * Otmizadores : 1 de cada 'família'   (Nadam, RMSProp, SGD, Ftrl)
# * Bactch size :  32 e 64
# * val e test size : .2 e .33 
# * learning rate : '0.0001' e '0.001' e '0.00001'
# * Dropout Rate: 0, 0.1, 0.2
# * Numero de camadas convolucionais 1 e 2
# * Numero de filtros em cada camada 10 , 32 , 64

In [ ]:
# parametros:
BATCH_SIZE = 64
EPOCHS = 10
VAL_AND_TST_SIZE = 0.2
# LEARN_RATE = 0.0001
# ******
# BETA_1 = 0.09
# BETA_2 = 0.0999
# EPSILON = 1e-07
# *****
EMBEDDING_DIMENSION = 300  
MAX_TEXT_SIZE = max_text_length 
VOCAB_SIZE = text_vocab_size

In [ ]:
def search_optimize (hp):

  hp_optimizer = hp.Choice('Optimizer', values=['Nadam', 'RMSprop', 'SGD', 'Ftrl'])

  if hp_optimizer == 'Nadam':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    # beta_1 = hp.Choice('beta1', values=[9e-1, 99e-2])
    # beta_2 = hp.Choice('beta2', values=[999e-3, 9999e-4])
    # epsilon = hp.Choice('epsilon', values=[1e-07, 1e-08])
  elif hp_optimizer == 'RMSprop':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    # nesterov = hp.Choice('nesterov', values=[True, False])
    # rho = hp.Choice('rho', values=[9e-1, 99e-2])
    # momentum = hp.Choice('momentum', values=[0.0, 1e-4])
    # epsilon = hp.Choice('epsilon', values=[1e-07, 1e-08])
    # centered = hp.Choice('nesterov', values=[True, False])
  elif hp_optimizer == 'SGD':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    # momentum = hp.Choice('momentum', values=[0.0, 1e-4])
    # nesterov = hp.Choice('nesterov', values=[True, False])
  elif hp_optimizer == 'Ftrl':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    # learning_rate_power = hp.Choice('learning_rate', values=[-0.5 , -0.2])
    # initial_accumulator_value = hp.Choice('learning_rate', values=[0.1, 0.01])
    # l1_regularization_strength = hp.Choice('learning_rate', values=[0.1, 0.01])
    # l2_regularization_strength = hp.Choice('learning_rate', values=[0.1, 0.01])
    # l2_shrinkage_regularization_strength = hp.Choice('learning_rate', values=[0.01, 0.001, 0.001])
    # beta = hp.Choice('learning_rate', values=[0.0001, 0.001])
  
 
  
  return hp_optimizer

def search_create_cnn (hp, vocab_size, embedding_dimen, max_text_size ):

  modelo = Sequential()

  # modelo.add(keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION, input_length=max_text_size))      
  # modelo.add(keras.layers.Input(shape=(8459, 55)))
  
  for i in range(hp.Int('Numero_de_Conv1d', min_value=1, max_value=3)):
    hp_padding=hp.Choice('padding_'+ str(i), values=['valid', 'same'])
    hp_filters=hp.Choice('filters_'+ str(i), values=[10, 32, 64])
    modelo.add(keras.layers.Conv1D(hp_filters,3, padding=hp_padding, activation="relu"))
  
  modelo.add(keras.layers.Flatten(input_shape=(max_text_size,)))
  
  for i in range(hp.Int('Numero_de_Camadas_densas',  min_value=1, max_value=3)):
    modelo.add(Dense(units=hp.Int('dense_' + str(i), min_value=MAX_TEXT_SIZE, max_value=(MAX_TEXT_SIZE*3), step=MAX_TEXT_SIZE), activation='relu'))
    modelo.add(Dropout(hp.Choice('dropout_'+ str(i), values=[0.0, 0.1, 0.2, 0.3])))            
                
  modelo.add(keras.layers.Dense(1, activation='sigmoid'))
  
 
  
  return modelo

def search_train_model(input_data, output_label, embedding_dimen, batch_size, epochs, validation_and_test_size):
   
    hp = keras_tuner.HyperParameters()

    x_train, x_test, y_train, y_test = train_test_split(input_data, output_label, test_size=validation_and_test_size, random_state=42)

    optimizer = search_optimize(hp)
    
    model = search_create_cnn(hp, VOCAB_SIZE, EMBEDDING_DIMENSION, MAX_TEXT_SIZE)
   
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=optimizer, metrics=['accuracy'])        
    
    ra_val = RocAucEvaluation(validation_data=(x_test, y_test), interval = 1)
    
    csv_logger = CSVLogger('log.csv', append=False, separator=';')                                                                                       
    
    print(x_train.shape)

    print("TO AQUI 0")
    tuner_mlp = keras_tuner.tuners.BayesianOptimization(model, seed=42, objective='val_loss', max_trials=30, directory='.', project_name='tuning-cnn')
    print("TO AQUI 1")
    
    hp_batch_size=hp.Choice('batch_size_', values=[32, 64])
   
    print("Estou no Search")
    tuner_mlp.search(x_train, y_train, batch_size=hp_batch_size, epochs=EPOCHS, validation_split = validation_and_test_size, verbose=2, use_multiprocessing=True,  callbacks = [ra_val, csv_logger])

In [ ]:
search_train_model(input_data, output_label, EMBEDDING_DIMENSION, BATCH_SIZE, EPOCHS, VAL_AND_TST_SIZE)

(8459, 55)
TO AQUI 0


TypeError: ignored

In [ ]:
def optimize (hp):

  hp_optimizer = hp.Choice('Optimizer', values=['Nadam', 'RMSprop', 'SGD', 'Ftrl'])

  if hp_optimizer == 'Nadam':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    beta_1 = hp.Choice('beta1', values=[9e-1, 99e-2])
    beta_2 = hp.Choice('beta2', values=[999e-3, 9999e-4])
    epsilon = hp.Choice('epsilon', values=[1e-07, 1e-08])
  elif hp_optimizer == 'RMSprop':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    nesterov = hp.Choice('nesterov', values=[True, False])
    rho = hp.Choice('rho', values=[9e-1, 99e-2])
    momentum = hp.Choice('momentum', values=[0.0, 1e-4])
    epsilon = hp.Choice('epsilon', values=[1e-07, 1e-08])
    centered = hp.Choice('nesterov', values=[True, False])
  elif hp_optimizer == 'SGD':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    momentum = hp.Choice('momentum', values=[0.0, 1e-4])
    nesterov = hp.Choice('nesterov', values=[True, False])
  elif hp_optimizer == 'Ftrl':
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    learning_rate_power = hp.Choice('learning_rate', values=[-0.5 , -0.2])
    initial_accumulator_value = hp.Choice('learning_rate', values=[0.1, 0.01])
    l1_regularization_strength = hp.Choice('learning_rate', values=[0.1, 0.01])
    l2_regularization_strength = hp.Choice('learning_rate', values=[0.1, 0.01])
    l2_shrinkage_regularization_strength = hp.Choice('learning_rate', values=[0.01, 0.001, 0.001])
    beta = hp.Choice('learning_rate', values=[0.0001, 0.001])
  
  return hp_optimizer

def create_cnn (hp, vocab_size, embedding_dimen, max_text_size ):

  modelo = Sequential()

  

  modelo.add(keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION, input_length=max_text_size))      
  
  for i in range(hp.Int('Numero_de_Conv1d', min_value=1, max_value=3)):
    hp_padding=hp.Choice('padding_'+ str(i), values=['valid', 'same'])
    hp_filters=hp.Choice('filters_'+ str(i), values=[10, 32, 64])
    modelo.add(keras.layers.Conv1D(hp_filters,(3, 3), padding=hp_padding, activation="relu"))
  
  modelo.add(keras.layers.Flatten(input_shape=(max_text_size,)))
  
  for i in range(hp.Int('Numero_de_Camadas_densas',  min_value=1, max_value=3)):
    modelo.add(Dense(units=hp.Int('dense_' + str(i), min_value=MAX_TEXT_SIZE, max_value=(MAX_TEXT_SIZE*3), step=MAX_TEXT_SIZE), activation='relu'))
    modelo.add(Dropout(hp.Choice('dropout_'+ str(i), values=[0.0, 0.1, 0.2, 0.3])))            
                
  modelo.add(keras.layers.Dense(1, activation='sigmoid'))
  
  return modelo
  
def train_model(input_data, output_label, embedding_dimen, batch_size, epochs, validation_and_test_size):
   
    hp = keras_tuner.HyperParameters()

    x_train, x_test, y_train, y_test = train_test_split(input_data, output_label, test_size=validation_and_test_size, random_state=42)

    

    optimizer = optimize(hp, LEARN_RATE, BETA_1, BETA_2, EPSILON)
    
    model = create_cnn(hp, VOCAB_SIZE, EMBEDDING_DIMENSION, MAX_TEXT_SIZE)
    
    model.summary()
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=optimizer, metrics=['accuracy'])        

    ra_val = RocAucEvaluation(validation_data=(x_test, y_test), interval = 1)
    csv_logger = CSVLogger('log.csv', append=False, separator=';')                                                                                       
    
    # print("Estou no Fit")
    # history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split = validation_and_test_size, verbose=2, use_multiprocessing=True,  callbacks = [ra_val, csv_logger])
    print("Estou no Search")
    tuner_mlp = keras_tuner.tuners.BayesianOptimization(model, seed=42, objective='val_loss', max_trials=30, directory='.', project_name='tuning-cnn')
    tuner_mlp.search(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split = validation_and_test_size, verbose=2, use_multiprocessing=True,  callbacks = [ra_val, csv_logger])
    
    
    
    # print("Estou no evaluate")
    # scores = model.evaluate(x_test, y_test, verbose=1)
    
    return history, model, x_test, y_test, ra_val, scores, csv_logger

In [ ]:
# plot dos gráficos
history, model1, x_test, y_test, ra_val, scores, csv_logger = train_model(input_data, output_label, EMBEDDING_DIMENSION, BATCH_SIZE, EPOCHS, VAL_AND_TST_SIZE)

In [ ]:
best_mlp_hyperparameters = tuner_mlp.get_best_hyperparameters(1)[0]
print("Best Hyper-parameters")
best_mlp_hyperparameters.values

In [ ]:
y_prob = model1.predict(x_test) 

y_labelpred = y_prob.round()

y_labeltrue=y_test

In [ ]:
# Parâmetros
NOME_REDE = REDE_CNN
NOME_BASE = BASE_2
NOME_FUNCAO = FUNCAO_ATV_SIGMOID
NOME_OTIMIZADOR = OTIMIZADOR_1

train_resume = json.dumps(pd.read_csv('log.csv',sep=';').to_dict()) 

roc_curve =  json.dumps({
    "false_positive": list(ra_val.false_positive_rate),
    "true_positive": list(ra_val.true_positive_rate),
    "score": ra_val.score
})

confusion = confusion_matrix(y_labeltrue,y_labelpred)
confusion_dict = json.dumps({
    "00": int(confusion[0][0]),
    "01": int(confusion[0][1]),
    "10": int(confusion[1][0]),
    "11": int(confusion[1][1])
})

# Salvando resultado do modelo
registro_resultado = {resultados.columns[0]: NOME_REDE, 
                      resultados.columns[1]: NOME_BASE, 
                      resultados.columns[2]: NOME_FUNCAO, 
                      resultados.columns[3]: NOME_OTIMIZADOR, 
                      resultados.columns[4]: scores[1],
                      resultados.columns[5]: roc_curve,
                      resultados.columns[6]: train_resume,
                      resultados.columns[7]: confusion_dict}

resultados = resultados.append(registro_resultado, ignore_index=True)
resultados

,rede,base,funcao_ativacao,otimizador,acuracia,roc_curve,train_resume,confusion_matrix
0,CNN,BASE 2,SIGMOID,1,0.919622,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 954, ""01"": 93, ""10"": 77, ""11"": 991}"
1,CNN,BASE 2,SIGMOID,1,0.925768,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 983, ""01"": 64, ""10"": 93, ""11"": 975}"
2,CNN,BASE 2,SIGMOID,1,0.928605,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 970, ""01"": 77, ""10"": 74, ""11"": 994}"
3,CNN,BASE 2,SIGMOID,1,0.921513,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 952, ""01"": 95, ""10"": 71, ""11"": 997}"
4,CNN,BASE 2,SIGMOID,1,0.927187,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 974, ""01"": 73, ""10"": 81, ""11"": 987}"
5,CNN,BASE 2,SIGMOID,1,0.908274,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 960, ""01"": 87, ""10"": 107, ""11"": 961}"
6,CNN,BASE 2,SIGMOID,1,0.930024,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 987, ""01"": 60, ""10"": 88, ""11"": 980}"
7,CNN,BASE 2,SIGMOID,1,0.930024,"{""false_positive_rate"": [0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 987, ""01"": 60, ""10"": 88, ""11"": 980}"
8,CNN,BASE 2,SIGMOID,1,0.922459,"{""false_positive"": [0.0, 0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 986, ""01"": 61, ""10"": 103, ""11"": 965}"
9,CNN,BASE 2,SIGMOID,1,0.911111,"{""false_positive"": [0.0, 0.0, 0.0, 0.0, 0.0, 0...","{""epoch"": {""0"": 0, ""1"": 1, ""2"": 2, ""3"": 3, ""4""...","{""00"": 964, ""01"": 83, ""10"": 105, ""11"": 963}"


In [ ]:
# Caso queira deletar uma linha use o código abaixo
# Use a propriedade label para especificar o índice da linha
#resultados = resultados.drop(labels=1, axis=0)
#resultados = resultados.reset_index(drop=True)
#resultados

In [ ]:
# Salva arquivo de saída
resultados.to_csv(PATH_ARQ_SAIDA)